In [1]:
# Import libraries

import os
import requests
from io import StringIO

import psycopg2
from psycopg2 import sql

# Data Manipulation
import pandas as pd
import geopandas as gpd

# Getting .env information (API keys, etc.)
from dotenv import load_dotenv

In [2]:
# Get REDCap token

load_dotenv()

token = os.getenv('REDCAP_TOKEN')

# Get Database credentials

creds = [os.getenv('DB_NAME'),
         os.getenv('DB_USER'),
         os.getenv('DB_PASS'),
         os.getenv('DB_PORT'),
         os.getenv('DB_HOST')
        ]

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))    

In [3]:
data = {
    'token': token,
    'content': 'record',
    'action': 'export',
    'format': 'csv',
    'type': 'flat',
    'csvDelimiter': '',
    'rawOrLabel': 'raw',
    'rawOrLabelHeaders': 'raw',
    'exportCheckboxLabel': 'false',
    'exportSurveyFields': 'false',
    'exportDataAccessGroups': 'false',
    'returnFormat': 'csv'
}
r = requests.post('https://redcap.ahc.umn.edu/api/',data=data)

# r.text

In [5]:
print('HTTP Status: ' + str(r.status_code))
df = pd.read_csv(StringIO(r.text))

HTTP Status: 200


In [6]:
# Spatialize dataframe

gdf = gpd.GeoDataFrame(df, 
                        geometry = gpd.points_from_xy(
                                    df.lon,
                                    df.lat,
                                    crs = 'EPSG:4326')
                               ).to_crs('EPSG:26915')

gdf['wkt'] = gdf.geometry.apply(lambda x: x.wkt)

In [7]:
# Upload to database

# Prep

final_df = gdf[['record_id', 'wkt']]

cols_for_db = ['record_id', 'geometry']

In [10]:
# Connect to PostGIS Database

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

conn = psycopg2.connect(**pg_connection_dict)

In [11]:
#connect to the cursor
cur = conn.cursor()

# iterate over the dataframe and insert each row into the database using a SQL INSERT statement

for index, row in final_df.iterrows():

    q1 = sql.SQL('INSERT INTO "Sign Up Information" ({}) VALUES ({},{});').format(
     sql.SQL(', ').join(map(sql.Identifier, cols_for_db)),
     sql.SQL(', ').join(sql.Placeholder() * (len(cols_for_db)-1)),
     sql.SQL('ST_Transform(ST_SetSRID(ST_GeomFromText(%s), 26915),4326)::geometry'))
    # print(q1.as_string(conn))
    # print(row)
    # break
    
    cur.execute(q1.as_string(conn),
        (list(row.values))
        )
    # Commit command

    conn.commit()

# Close the cursor and connection
cur.close()
conn.close()